In [ ]:
import pandas as pd
import seaborn as sns
import json
import matplotlib.pyplot as plt
import numpy as np
import re

## Understanding Data

In [ ]:
with open("appraisals_dataset.json","r") as f:
  data = json.load(f)["appraisals"][0]

subject_property = data["subject"]
potential_comps_list = data["comps"]
properties_data_list = data["properties"]

dataset = pd.DataFrame(properties_data_list)

In [ ]:
dataset.head(10)

,id,address,bedrooms,gla,city,province,postal_code,property_sub_type,structure_type,style,...,year_built,roof,basement,cooling,heating,close_price,close_date,public_remarks,latitude,longitude
0,367,463 Conservatory Dr,3,1500.0,Kingston,Ontario,K7M 9C8,Detached,Detached,"Brick, Vinyl Siding",...,NaN,None,Unfinished,Central Air,Forced Air,674000.0,2025-01-13,Welcome to this beautifully maintained and spa...,44.2325,-76.5901
1,163443,463 Conservatory Drive,3,1750.0,Kingston,Ontario,K7M 9C8,Detached,"Detached, 2-Storey",2-Storey,...,NaN,,Unfinished,Central Air,Forced Air,674000.0,2025-01-13,Welcome to this beautifully maintained and spa...,44.2325,-76.5901
2,378,311 Janette St,3,1500.0,Kingston,Ontario,K7P 0K8,Freehold Townhouse,Freehold Townhouse,2-Storey,...,NaN,None,Fin W/O,Central Air,Forced Air,585000.0,2025-01-14,"Come and see what this fully finished, Barr co...",44.2622,-76.5904
3,130023,311 Janette Street,3,1300.0,Kingston,Ontario,K7P 0K8,Freehold Townhouse,"Freehold Townhouse, 2-Storey",2-Storey,...,NaN,,Finished with Walk-Out,Central Air,Forced Air,585000.0,2025-01-14,"Come and see what this fully finished, Barr co...",44.2622,-76.5904
4,2782,4056 Bath Rd,4,NaN,Kingston,Ontario,K7M 4Y4,Rural Resid,Rural Resid,2-Storey,...,NaN,None,"Part Bsmt, Unfinished",None,Baseboard,685000.0,2025-01-15,"Current valuation for 4056 Bath Rd, South of T...",44.2407,-76.6102
5,2783,786 HIGH GATE PARK Dr,3,1300.0,Kingston,Ontario,K7M 5Z8,Detached,Detached,Two,...,NaN,None,Finished,None,Forced Air,495000.0,2025-01-15,"WELCOME TO 786 HIGH GATE PARK DRIVE, LOCATED I...",44.2489,-76.6045
6,138739,786 HIGH GATE PARK Drive,3,1300.0,Kingston,Ontario,K7M 5Z8,Detached,"Detached, Sidesplit 3",Sidesplit 3,...,NaN,,Finished,,Forced Air,495000.0,2025-01-15,"WELCOME TO 786 HIGH GATE PARK DRIVE, LOCATED I...",44.2489,-76.6045
7,2763,784 Downing St,5,1250.0,Kingston,Ontario,K7M 5N2,Detached,Detached,Sidesplit 4,...,NaN,None,Unfinished,Central Air,Forced Air,567500.0,2025-01-17,Welcome to 784 Downing St located in the charm...,44.2492,-76.5968
8,146595,784 Downing Street,4,1300.0,Kingston,Ontario,K7M 5N2,Detached,"Detached, Sidesplit 4",Sidesplit 4,...,NaN,,Unfinished,Central Air,Forced Air,567500.0,2025-01-17,Welcome to 784 Downing St located in the charm...,44.2492,-76.5968
9,163,593 Roosevelt Dr,4,1880.0,Kingston,Ontario,K7M 8T7,Detached,Detached,Bungalow,...,NaN,None,Finished,Central Air,Forced Air,681000.0,2025-01-21,Welcome to 593 Roosevelt Drive! Nestled in the...,44.2346,-76.5871


In [ ]:
#Summary statistics of uncleaned data
print(dataset.info())

print("\nDecribing the data:")
print(dataset.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146 entries, 0 to 145
Data columns (total 28 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        146 non-null    int64  
 1   address                   146 non-null    object 
 2   bedrooms                  146 non-null    int64  
 3   gla                       139 non-null    float64
 4   city                      146 non-null    object 
 5   province                  146 non-null    object 
 6   postal_code               146 non-null    object 
 7   property_sub_type         142 non-null    object 
 8   structure_type            145 non-null    object 
 9   style                     144 non-null    object 
 10  levels                    144 non-null    object 
 11  room_count                146 non-null    int64  
 12  full_baths                38 non-null     float64
 13  half_baths                7 non-null      float64
 14  main_level

In [ ]:
duplicated_rows = dataset.duplicated().sum()
if(duplicated_rows>0):
  print("The duplicated values that are dropped are:", duplicated_rows)
  dataset.drop_duplicates(inplace=True)


## Data Cleaning

In [ ]:
dataset = dataset.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
dataset = dataset.drop(["id","city","province","bg_fin_area","main_level_finished_area","upper_lvl_fin_area", "public_remarks"],axis=1)

In [ ]:
# Convert to integer
int_cols = ["full_baths", "half_baths", "year_built"]
dataset[int_cols] = dataset[int_cols].astype("Int64")

# Convert to float (already correct but ensure nullable floats)

dataset["id"] = dataset["id"].astype("Int64")

float_cols = [
    "gla", "close_price", "latitude", "longitude", "lot_size_sf"
]
dataset[float_cols] = dataset[float_cols].astype("Float64")

# Object columns that should remain as string/categorical
string_cols = [
    "address", "city", "province", "postal_code", "property_sub_type",
    "structure_type", "style", "levels", "roof", "basement",
    "cooling", "heating", "public_remarks"
]
dataset[string_cols] = dataset[string_cols].astype("string")

# Convert close_date to datetime
dataset["close_date"] = pd.to_datetime(dataset["close_date"], errors="coerce")
dataset.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146 entries, 0 to 145
Data columns (total 28 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   id                        146 non-null    Int64         
 1   address                   146 non-null    string        
 2   bedrooms                  146 non-null    int64         
 3   gla                       139 non-null    Float64       
 4   city                      146 non-null    string        
 5   province                  146 non-null    string        
 6   postal_code               146 non-null    string        
 7   property_sub_type         142 non-null    string        
 8   structure_type            145 non-null    string        
 9   style                     144 non-null    string        
 10  levels                    144 non-null    string        
 11  room_count                146 non-null    int64         
 12  full_baths            

In [ ]:
address_map = {
    'St': 'Street',
    'Rd': 'Road',
    'Ave': 'Avenue',
    'Cres': 'Crescent',
    'Circ': 'Circle',
    'Blvd': 'Boulevard',
    'Pl': 'Place',
    'Dr': 'Drive',
    'Ln': 'Lane',
    'Ter': 'Terrace',
    'W': '',
    'S': ''
}

def standardize_address(addr):
    # Normalize case
    addr = addr.title()

    # Replace abbreviations using word boundaries
    for abbr, full in address_map.items():
        addr = re.sub(rf'\b{re.escape(abbr)}\b', full, addr)

    # Clean up extra whitespace
    addr = re.sub(r'\s+', ' ', addr).strip()
    return addr


dataset["address"] = dataset["address"].apply(standardize_address)

# Preview
dataset['address'].unique()


array(['463 Conservatory Drive', '311 Janette Street', '4056 Bath Road',
       '786 High Gate Park Drive', '784 Downing Street',
       '593 Roosevelt Drive', '657 Gwen Avenue', '1341 Tremont Drive',
       'Unit 402 - 649 Davis Drive', '1028 Bauder Crescent',
       '1040 Earnhart Street', '994 Westminster Place',
       '2138 Balantrae Circle', '692 Truedell Road',
       '728 Sussex Boulevard', 'Unit 51 - 808 Datzell Lane',
       '1126 Dunham Street', '983 Westminster Place', '4061 Bath Road',
       '661 Roosevelt Drive', '657 Barnsley Crescent',
       '418 Conservatory Drive', '1001 Lombardy Street',
       '605 Truedell Road', 'Unit 108 - 835 Milford Drive',
       '771 Ashwood Drive', '494 Roosevelt Drive', '393 Bernice Drive',
       '871 Crestwood Avenue', '760 Grouse Crescent', '698 Fleet Street',
       '912 Oakview Street', '995 Amberdale Crescent',
       '616 Pimlico Place', '792 Safari Drive', '667 Truedell Road',
       '995 Waterbury Crescent', 'Unit 305 - 675 Davis

In [ ]:
# Mapping to standard values
property_sub_type_map = {
    'Detached': 'Detached',
    'Single Family Residence': 'Detached',
    'Freehold Townhouse': 'Townhouse',
    'Condo Townhouse': 'Townhouse',
    'Condo Apt': 'Condo Apartment',
    'Condo Apartment': 'Condo Apartment',
    'Common Element Condo': 'Condo (Common Element)',
    'Rural Resid': 'Rural Residential',
    'Semi-Detached': 'Semi-Detached',
    'Duplex': 'Duplex'
}

# Apply the mapping
dataset["property_sub_type"] = dataset["property_sub_type"].replace(property_sub_type_map)

# Handle missing values
dataset["property_sub_type"] = dataset["property_sub_type"].fillna('Unknown')
dataset["property_sub_type"].unique()

<StringArray>
[              'Detached',              'Townhouse',      'Rural Residential',
        'Condo Apartment',          'Semi-Detached',                'Unknown',
 'Condo (Common Element)',                 'Duplex']
Length: 8, dtype: string

In [ ]:
# Mapping of structure roots to standardized types
structure_map = {
    'Detached': 'Detached',
    'Freehold Townhouse': 'Townhouse',
    'Condo Townhouse': 'Townhouse',
    'Condo Apartment': 'Condo Apartment',
    'Condo Apt': 'Condo Apartment',
    'Apartment': 'Apartment',
    'Semi-Detached': 'Semi-Detached',
    'Rural Resid': 'Rural Residential',
    'Common Element Condo': 'Condo (Common Element)',
    'Single Family Residence': 'Detached',
    'Duplex': 'Duplex'
}

# Function to standardize structure type
def standardize_structure(s):
    if pd.isna(s):
        return 'Unknown'

    # Extract the leading type before comma
    main_type = s.split(',')[0].strip()

    # Map to standardized form
    return structure_map.get(main_type, main_type)  # fallback to original if not mapped

# Apply function
dataset["structure_type"] = dataset["structure_type"].apply(standardize_structure).astype('category')

# View result
dataset["structure_type"].unique()

['Detached', 'Townhouse', 'Rural Residential', 'Apartment', 'Condo Apartment', 'Semi-Detached', 'Unknown', 'Condo (Common Element)', 'Duplex']
Categories (9, object): ['Apartment', 'Condo (Common Element)', 'Condo Apartment', 'Detached', ...,
                         'Rural Residential', 'Semi-Detached', 'Townhouse', 'Unknown']

In [ ]:
# Mapping to standardized styles
style_map = {
    'Brick, Vinyl Siding':'Vinyl Siding',
    '2-Storey': '2-Storey',
    'Two': '2-Storey',
    '1 1/2 Storey': '1.5-Storey',
    '3-Storey': '3-Storey',
    'Bungalow': 'Bungalow',
    'Bungalow-Raised': 'Raised Bungalow',
    'Apartment': 'Apartment',
    'Bachelor/Studio': 'Studio',
    'Sidesplit': 'Sidesplit',
    'Sidesplit 3': 'Sidesplit',
    'Sidesplit 4': 'Sidesplit',
    'Backsplit 3': 'Backsplit',
    'Backsplit 4': 'Backsplit',
    'Other': 'Other'
}

# Function to standardize
def standardize_style(value):
    if pd.isna(value):
        return 'Unknown'

    # If compound (e.g. 'Brick, Vinyl Siding'), drop it — irrelevant to style
    if ',' in value:
        return 'Unknown'

    # Return mapped value or 'Unknown'
    return style_map.get(value.strip(), 'Unknown')

# Apply function
dataset["style"] = dataset["style"].apply(standardize_style).astype('category')

dataset["style"].unique()

['Unknown', '2-Storey', 'Sidesplit', 'Bungalow', 'Apartment', ..., 'Backsplit', 'Studio', '3-Storey', '1.5-Storey', 'Other']
Length: 11
Categories (11, object): ['1.5-Storey', '2-Storey', '3-Storey', 'Apartment', ..., 'Raised Bungalow',
                          'Sidesplit', 'Studio', 'Unknown']

In [ ]:
# Mapping to clean categories
levels_map = {
    'Two': '2-Storey',
    '2-Storey': '2-Storey',
    'One': '1-Storey',
    '3.0': '3-Storey',
    '3-Storey': '3-Storey',
    '1 1/2 Storey': '1.5-Storey',
    'Bungalow': 'Bungalow',
    'Bungalow-Raised': 'Raised Bungalow',
    'Apartment': 'Apartment',
    'Bachelor/Studio': 'Studio',
    'Sidesplit': 'Sidesplit',
    'Sidesplit 3': 'Sidesplit',
    'Sidesplit 4': 'Sidesplit',
    'Backsplit 3': 'Backsplit',
    'Backsplit 4': 'Backsplit',
    'Other': 'Other'
}

# Function to standardize
def standardize_levels(value):
    if pd.isna(value):
        return 'Unknown'
    return levels_map.get(value.strip(), 'Unknown')

# Apply mapping
dataset["levels"] = dataset["levels"].apply(standardize_levels).astype('category')

dataset["levels"].unique()

['2-Storey', 'Unknown', 'Sidesplit', '1-Storey', 'Bungalow', ..., '3-Storey', 'Backsplit', 'Studio', '1.5-Storey', 'Other']
Length: 12
Categories (12, object): ['1-Storey', '1.5-Storey', '2-Storey', '3-Storey', ..., 'Raised Bungalow',
                          'Sidesplit', 'Studio', 'Unknown']

In [ ]:
roof_map = {
    'Asphalt Shing': 'Asphalt Shingle',
    '': pd.NA  # Treat empty string as missing
}

# Convert to string type (to safely modify values)
dataset["roof"] = dataset["roof"].astype("string")

# Strip whitespace and normalize values
dataset["roof"] = dataset["roof"].str.strip()

# Apply the map
roof_cleaned = dataset["roof"].replace(roof_map)
roof_cleaned = dataset["roof"].replace("",pd.NA)

# Fill remaining NA values with a label (or use mode)
dataset["roof"] = dataset["roof"].fillna("Unknown")

# Optional: convert to category for modeling
dataset["roof"] = dataset["roof"].astype("category")
dataset['roof'].unique()

['Unknown', '', 'Asphalt Shing']
Categories (3, string): [, Asphalt Shing, Unknown]

In [ ]:
def standardize_basement(value):
    if pd.isna(value) or value.strip() == "":
        return "Unknown"

    value = value.lower()

    # Simplify and map to core categories
    if "none" in value:
        return "None"
    if "unfinished" in value and "partial" not in value:
        return "Unfinished"
    if "part fin" in value or "partial" in value or "partially" in value:
        return "Partially Finished"
    if "finished" in value:
        return "Finished"
    if "apartment" in value:
        return "Apartment"
    if "crawl" in value:
        return "Crawl Space"
    if "walk-out" in value or "w/o" in value:
        return "Walk-Out"
    if "walk-up" in value:
        return "Walk-Up"
    if "sep entrance" in value or "separate entrance" in value:
        return "Separate Entrance"
    if "full" in value:
        return "Full"
    if "other" in value:
        return "Other"

    return "Unknown"

dataset['basement'] = dataset['basement'].apply(standardize_basement).astype("category")
dataset['basement'].unique()

['Unfinished', 'Walk-Out', 'Finished', 'Unknown', 'Partially Finished', 'Apartment', 'Walk-Up', 'Other', 'None', 'Full']
Categories (10, object): ['Apartment', 'Finished', 'Full', 'None', ..., 'Unfinished', 'Unknown',
                          'Walk-Out', 'Walk-Up']

In [ ]:
# Normalize values
cooling_map = {
    'Window Unit(s)': 'Window Unit',
    'Wall Unit(s)': 'Wall Unit',
    '': pd.NA
}

dataset["cooling"] = dataset["cooling"].replace(cooling_map)

# Fill missing
dataset["cooling"] = dataset["cooling"].fillna("Unknown")

# Convert to category
dataset["cooling"] = dataset["cooling"].astype("category")

dataset['cooling'].unique()

['Central Air', 'Unknown', 'Window Unit', 'Wall Unit']
Categories (4, string): [Central Air, Unknown, Wall Unit, Window Unit]